# ***Introducing TorchFlare***

![logo](https://raw.githubusercontent.com/Atharva-Phatak/torchflare/main/docs/static/images/TorchFlare.gif)


***TorchFlare*** is a simple, beginner-friendly and an easy-to-use PyTorch Framework train your models without much effort.
It provides an almost Keras-like experience for training
your models with all the callbacks, metrics, etc


### ***Features***
* _A high-level module for Keras-like training._
* _Off-the-shelf Pytorch style Datasets/Dataloaders for standard tasks such as **Image classification, Image segmentation,
  Text Classification**, etc_
* _**Callbacks** for model checkpoints, early stopping, and much more!_
* _**Metrics** and much more._
* _**Reduction** of the boiler plate code required for training your models._

***
* **Github**: https://github.com/Atharva-Phatak/torchflare
* **Docs**: https://atharva-phatak.github.io/torchflare/

**Show some love to [TorchFlare](https://github.com/Atharva-Phatak/torchflare) by giving start if you like the library.**

**If you find bug or have feature requests or create awesome kernels/notebooks using TorchFlare, open up a issue in repo.**

In [ ]:
!pip install -qq torchflare

* For some reason installation takes some time, I guess because torchflare is based on latest versions. 

In [ ]:
!pip install -qq nb_black

In [ ]:
%load_ext lab_black

#### ***Importing Libraries***

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd

import torchflare.callbacks as cbs
import torchflare.metrics as metrics
from torchflare.experiments import Experiment
from torchflare.datasets import TextDataloader

import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")

#### ***Downloading our tokenizer.***

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained(
    "bert-base-uncased", do_lower_case=True
)

In [ ]:
train_df = pd.read_csv("../input/commonlitreadabilityprize/train.csv")

In [ ]:
train_df.head()

#### ***Using simple train_test_split for data splitting.***
* Will add another kernel showing how to do crossval with torchflare.

In [ ]:
train_df, valid_df = train_test_split(train_df, test_size=0.3, random_state=42)

* ***Using TextDataloader functionality from TorchFlarefor easy creation of text dataloaders.***

In [ ]:
train_dl = TextDataloader.from_df(
    df=train_df,
    tokenizer=tokenizer,
    max_len=185,
    input_col="excerpt",
    label_cols="target",
).get_loader(batch_size=32, shuffle=True)

valid_dl = TextDataloader.from_df(
    df=valid_df,
    tokenizer=tokenizer,
    max_len=185,
    input_col="excerpt",
    label_cols="target",
).get_loader(batch_size=64, shuffle=False)

* ***Defining BERT Model.***

In [ ]:
class BERTClass(nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert = transformers.BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, 1)

    def forward(self, input_ids, token_type_ids, attention_mask):
        _, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=False,
        )
        output = self.fc(output)
        return output

In [ ]:
model = BERTClass()

* ***Defining the loss function.***

In [ ]:
def rmse(op, y):
    return torch.sqrt(nn.MSELoss()(op, y.float()))

* ***Defining Callbacks and metrics and some optimizer params.***

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
callbacks = [
    cbs.ModelCheckpoint(monitor="val_loss", mode="min", save_dir="./"),
    cbs.ReduceLROnPlateau(mode="min", patience=2),
]

#### ***Defining experiment in TorchFlare is easy***
1. First define some constant params like epochs , seeds, etc
2. Compile your experiment with model, optimizer, callbacks, etc
3. Fit your experiment on traininig and validation loaders.

In [ ]:
exp = Experiment(num_epochs=4, fp16=False, device="cuda", seed=42)

# Compiling the experiment
exp.compile_experiment(
    model=model,
    optimizer="AdamW",
    optimizer_params=dict(
        model_params=optimizer_parameters, lr=3e-4
    ),  # used model_params argument for custom optimizer params.
    callbacks=callbacks,
    criterion=rmse,
    metrics=None,
    main_metric="loss",
)

In [ ]:
exp.fit_loader(train_dl, valid_dl)

#### Let's take a experiment History. 

In [ ]:
exp.history

In [ ]:
print(f"Mean VAL RMSE  : {sum(exp.history['train_loss'])/len(exp.history['Epoch'])}")

* Note you wont be possibly be able to plot history this since we upgraded the Pillow version and Kaggle kernel has pip resolve issues(matplotlib compat issues). I will try update the requirements for torchflare to make it compatible with Kaggle Kernels. 


### ***Again this goes without saying, If you like this workflow and want to see more of it please star [TorchFlare](https://github.com/Atharva-Phatak/torchflare).***